In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발
- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다.
이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다.

- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다.
참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다.

- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [8]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb

# Load Data

In [3]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2024_생명연구자원_AI활용_경진대회/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2024_생명연구자원_AI활용_경진대회/test.csv")

In [ ]:
slicingtrain = train.iloc[:100,:]

In [ ]:
slicingtrain.to_csv('/content/drive/MyDrive/Colab Notebooks/2024_생명연구자원_AI활용_경진대회/slicingtrain.csv', encoding='UTF-8', index=False)

In [4]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


- WT는 Wild Type의 약자로, 자연 상태에서 존재하는 표준 또는 정상 유전형
- fs: "frameshift"를 의미하는데, 이는 돌연변이에 의해 단백질 번역의 읽기 프레임이 변화된 상태
- *: 종결 코돈

# EDA

## 암종별 변이된 유전체 칼럼의 수

In [4]:
data_columns = list(train.columns)

In [5]:
cancer = list(train["SUBCLASS"].unique())
cancer

['KIPAN',
 'SARC',
 'SKCM',
 'KIRC',
 'GBMLGG',
 'STES',
 'BRCA',
 'THCA',
 'LIHC',
 'HNSC',
 'PAAD',
 'OV',
 'PRAD',
 'UCEC',
 'LAML',
 'COAD',
 'ACC',
 'LGG',
 'LUSC',
 'LUAD',
 'CESC',
 'PCPG',
 'THYM',
 'BLCA',
 'TGCT',
 'DLBC']

In [7]:
data_distribution = {}
for i in range(len(cancer)):
    data_distribution[cancer[i]] = {}
data_distribution

{'KIPAN': {},
 'SARC': {},
 'SKCM': {},
 'KIRC': {},
 'GBMLGG': {},
 'STES': {},
 'BRCA': {},
 'THCA': {},
 'LIHC': {},
 'HNSC': {},
 'PAAD': {},
 'OV': {},
 'PRAD': {},
 'UCEC': {},
 'LAML': {},
 'COAD': {},
 'ACC': {},
 'LGG': {},
 'LUSC': {},
 'LUAD': {},
 'CESC': {},
 'PCPG': {},
 'THYM': {},
 'BLCA': {},
 'TGCT': {},
 'DLBC': {}}

In [8]:
from tqdm import tqdm
for i in tqdm(range(len(train))):
    for j in range(2, len(data_columns)):
        if train.iloc[i, j] != "WT":
            try:
                data_distribution[train.iloc[i,1]][data_columns[j]] += 1
            except:
                data_distribution[train.iloc[i,1]][data_columns[j]] = 1


100%|██████████| 6201/6201 [13:04<00:00,  7.91it/s]


In [9]:
len(data_distribution)

26

In [10]:
for i in range(len(data_distribution)):
    data_distribution[cancer[i]] = dict(sorted(data_distribution[cancer[i]].items(), key=lambda x: x[1], reverse=True))




In [11]:
def top_n(count_dict, n=5):
    return sorted(count_dict.items(), reverse=True, key=lambda x: x[1])[:n]

In [12]:
for x in cancer:
    print(top_n(data_distribution[x]))

[('VHL', 154), ('DST', 27), ('KMT2D', 26), ('MTOR', 25), ('TP53', 24)]
[('TP53', 52), ('ATRX', 25), ('PCLO', 14), ('RB1', 13), ('RYR1', 10)]
[('BRAF', 139), ('PCLO', 133), ('MXRA5', 105), ('MYH1', 91), ('MYH4', 90)]
[('VHL', 151), ('DST', 19), ('MTOR', 19), ('KMT2D', 17), ('PABPC1', 16)]
[('IDH1', 189), ('TP53', 169), ('ATRX', 99), ('PTEN', 75), ('EGFR', 67)]
[('TP53', 214), ('SYNE1', 116), ('PCLO', 91), ('RYR2', 88), ('SPTA1', 79)]
[('PIK3CA', 250), ('TP53', 218), ('CDH1', 73), ('GATA3', 58), ('MAP3K1', 56)]
[('BRAF', 186), ('TG', 12), ('HRAS', 12), ('RYR1', 6), ('DLC1', 5)]
[('TP53', 41), ('CTNNB1', 38), ('RYR2', 26), ('DST', 19), ('PCLO', 18)]
[('TP53', 147), ('CDKN2A', 48), ('PIK3CA', 47), ('SYNE1', 43), ('NOTCH1', 42)]
[('TP53', 79), ('CDKN2A', 19), ('RYR1', 12), ('GNAS', 9), ('PLEC', 9)]
[('TP53', 190), ('RYR2', 13), ('COL6A3', 11), ('EGFR', 10), ('NF1', 10)]
[('SPOP', 28), ('TP53', 14), ('SPTA1', 11), ('SYNE1', 10), ('KMT2D', 9)]
[('PTEN', 122), ('PIK3CA', 97), ('CTNNB1', 61), (

In [ ]:
train.loc[6153, 'ABCA3']

'D342N E506K P1301S'

- 많은 암종에서 TP53 유전체에서의 변이가 발견 됨.
- 다른 암종이 발생하였지만 유사한 유전체의 변이의 특성이 발견됨.
- 유전체 변이에서 fs 나 종결 코돈으로 변이된 비율을 확인 해 볼 필요성을 느낌.

## 동의돌연변이 거나 WT로만 구성되어있는 환자

In [13]:
data_distribution_same_amino = {}
for i in range(len(cancer)):
    data_distribution_same_amino[cancer[i]] = 0
data_distribution_same_amino

{'KIPAN': 0,
 'SARC': 0,
 'SKCM': 0,
 'KIRC': 0,
 'GBMLGG': 0,
 'STES': 0,
 'BRCA': 0,
 'THCA': 0,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 0,
 'OV': 0,
 'PRAD': 0,
 'UCEC': 0,
 'LAML': 0,
 'COAD': 0,
 'ACC': 0,
 'LGG': 0,
 'LUSC': 0,
 'LUAD': 0,
 'CESC': 0,
 'PCPG': 0,
 'THYM': 0,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

In [14]:
len(data_columns)

4386

In [14]:
genetic_variation_same_amino = []

In [15]:
for i in tqdm(range(len(train))):
    for j in range(2, len(data_columns)):
        flag_break=False
        if train.iloc[i, j] == "WT":
            continue
        else:
            if " " not in train.iloc[i, j]:
                if train.iloc[i, j][0] != train.iloc[i, j][-1]:
                    flag_break=True
                    break

            else:
                amino = train.iloc[i, j].split()
                ch = 0
                for k in range(len(amino)):

                    if amino[k][0] == amino[k][-1]:

                        ch += 1
                if (ch/len(amino)) != 1:
                    flag_break=True
                    break

    if flag_break:
        continue
    genetic_variation_same_amino.append(i)
    data_distribution_same_amino[train.iloc[i,1]] += 1


100%|██████████| 6201/6201 [01:37<00:00, 63.92it/s]


In [ ]:
genetic_variation_same_amino

[7,
 18,
 66,
 102,
 122,
 129,
 133,
 235,
 256,
 382,
 389,
 412,
 413,
 576,
 595,
 636,
 671,
 781,
 813,
 825,
 894,
 913,
 1007,
 1121,
 1386,
 1425,
 1460,
 1480,
 1555,
 1587,
 1753,
 1759,
 1786,
 1808,
 2032,
 2082,
 2216,
 2267,
 2284,
 2295,
 2317,
 2426,
 2514,
 2515,
 2528,
 2531,
 2545,
 2613,
 2628,
 2630,
 2639,
 2646,
 2776,
 2778,
 2794,
 2796,
 2823,
 3016,
 3055,
 3089,
 3093,
 3108,
 3258,
 3304,
 3324,
 3371,
 3469,
 3476,
 3583,
 3597,
 3607,
 3663,
 3668,
 3771,
 3789,
 3850,
 3904,
 3935,
 3960,
 4004,
 4044,
 4083,
 4107,
 4122,
 4310,
 4349,
 4389,
 4421,
 4467,
 4572,
 4579,
 4655,
 4658,
 4686,
 4788,
 4823,
 4879,
 4904,
 4941,
 4962,
 5079,
 5084,
 5114,
 5138,
 5193,
 5207,
 5291,
 5297,
 5345,
 5354,
 5510,
 5529,
 5625,
 5648,
 5708,
 5749,
 5798,
 5804,
 5972,
 6116,
 6134]

In [17]:
len(genetic_variation_same_amino)

121

- 1.95% 차지
- 유전체에 대해서 정상인 경우에서 암 발생됨.
- 유전자 변이 요소외의 외부 요인에 기인한 data
- 데이터 drop 결정

## 종결 코돈이나 FS로 변이된 수

In [ ]:
genetic_variation_fs_stop = []

In [ ]:
cancer

['KIPAN',
 'SARC',
 'SKCM',
 'KIRC',
 'GBMLGG',
 'STES',
 'BRCA',
 'THCA',
 'LIHC',
 'HNSC',
 'PAAD',
 'OV',
 'PRAD',
 'UCEC',
 'LAML',
 'COAD',
 'ACC',
 'LGG',
 'LUSC',
 'LUAD',
 'CESC',
 'PCPG',
 'THYM',
 'BLCA',
 'TGCT',
 'DLBC']

In [ ]:
data_distribution_fs_stop = {}
for i in range(len(cancer)):
    data_distribution_fs_stop[cancer[i]] = 0
data_distribution_fs_stop

{'KIPAN': 0,
 'SARC': 0,
 'SKCM': 0,
 'KIRC': 0,
 'GBMLGG': 0,
 'STES': 0,
 'BRCA': 0,
 'THCA': 0,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 0,
 'OV': 0,
 'PRAD': 0,
 'UCEC': 0,
 'LAML': 0,
 'COAD': 0,
 'ACC': 0,
 'LGG': 0,
 'LUSC': 0,
 'LUAD': 0,
 'CESC': 0,
 'PCPG': 0,
 'THYM': 0,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

In [ ]:
for i in tqdm(range(len(train))):
    for j in range(2, len(data_columns)):
        flag_break=False
        # 정상 유전체일 경우 다음 컬럼
        if train.iloc[i, j] == "WT":
            continue
        else:
        # 단일 변이일 경우
            if " " not in train.iloc[i, j]:

                if train.iloc[i, j][-1] == "*":
                    flag_break=True
                    break
                elif train.iloc[i, j][-2:] == "fs":
                    flag_break=True
                    break
        # 다중 변이일 경우
            else:
                amino = train.iloc[i, j].split()
                flag_break2=False
                for k in range(len(amino)):

                    if amino[k][-1] == "*":
                        flag_break2=True
                        break
                    elif amino[k][-2:] == "fs":
                        flag_break2=True
                        break


                if flag_break2:
                    flag_break=True
                    break

    if flag_break:
        genetic_variation_fs_stop.append(i)
        data_distribution_fs_stop[train.iloc[i,1]] += 1
        continue


100%|██████████| 6201/6201 [06:48<00:00, 15.19it/s]


In [ ]:
len(genetic_variation_fs_stop)

4578

- 73.83%에서 fs나 종결코돈의 유전변이 발생

## >가 포함된 환자

In [30]:
genetic_variation_same_amino2 = []
data_distribution_same_amino2 = {}
for i in range(len(cancer)):
    data_distribution_same_amino2[cancer[i]] = 0
data_distribution_same_amino2

{'KIPAN': 0,
 'SARC': 0,
 'SKCM': 0,
 'KIRC': 0,
 'GBMLGG': 0,
 'STES': 0,
 'BRCA': 0,
 'THCA': 0,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 0,
 'OV': 0,
 'PRAD': 0,
 'UCEC': 0,
 'LAML': 0,
 'COAD': 0,
 'ACC': 0,
 'LGG': 0,
 'LUSC': 0,
 'LUAD': 0,
 'CESC': 0,
 'PCPG': 0,
 'THYM': 0,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

In [31]:
for i in tqdm(range(len(train))):
    for j in range(2, len(data_columns)):
        flag_break=False
        if train.iloc[i, j] == "WT":
            continue
        else:
            if " " not in train.iloc[i, j]:
                if '>' in train.iloc[i, j]:

                    continue
                elif train.iloc[i, j][0] != train.iloc[i, j][-1]:
                    flag_break=True
                    break


            else:
                amino = train.iloc[i, j].split()
                ch = 0
                for k in range(len(amino)):

                    if amino[k][0] == amino[k][-1]:

                        ch += 1
                    elif '>' in amino[k]:
                        ch += 1
                if (ch/len(amino)) != 1:
                    flag_break=True
                    break

    if flag_break:
        continue
    genetic_variation_same_amino2.append(i)
    data_distribution_same_amino2[train.iloc[i,1]] += 1


100%|██████████| 6201/6201 [01:45<00:00, 58.88it/s]


In [32]:
len(genetic_variation_same_amino2)

121

## test data set 확인

In [19]:
t_genetic_variation_fs_stop = []


In [20]:
test.fillna("WT", inplace=True)

In [ ]:
for i in tqdm(range(len(test))):
    for j in range(1, len(data_columns)-1):
        flag_break=False
        # 정상 유전체일 경우 다음 컬럼
        if test.iloc[i, j] == "WT":
            continue
        else:
        # 단일 변이일 경우
            if " " not in test.iloc[i, j]:

                if test.iloc[i, j][-1] == "*":
                    flag_break=True
                    break
                elif test.iloc[i, j][-2:] == "fs":
                    flag_break=True
                    break
        # 다중 변이일 경우
            else:
                amino = test.iloc[i, j].split()
                flag_break2=False
                for k in range(len(amino)):

                    if amino[k][-1] == "*":
                        flag_break2=True
                        break
                    elif amino[k][-2:] == "fs":
                        flag_break2=True
                        break


                if flag_break2:
                    flag_break=True
                    break

    if flag_break:
        t_genetic_variation_fs_stop.append(i)
        continue


  0%|          | 3/2546 [00:00<01:51, 22.73it/s]

M267I M206I
T239S T220S
I3060V
Y1289H
G5E
Q117R K392E K324E K303E K662E K456E K705E
S221Kfs S205Kfs S262Kfs S169Kfs
R587Q
L306Yfs
K386N
V1444M
T71T
V443I
K758N
R1646*
S413F
L171L
A894A
D805G
R132H
N464S
E106K
R306*
G1317G
I752I
A449A
C340Y
F116fs
D625H D675H D775H
L404L
L15L
L491V L461V
A229P
S376T
Q804H
A496G
L548V L961V L982V L1022V
G102R
G46G
A218P A179P A178P A192P A219P
A1115P
H629D
V4V
X966S X1438S
C170S
A443G
R583T
P212A
G102G G41G
F255V
V1034L V1020L V947L V826L V963L V893L
T231R T212R
Q398H Q475H Q187H Q415H Q159H Q399H Q133H Q323H Q443H
Q326E
R76G
T23R
Y283X
A213G
R615R
L272V L270V
L340P
E354Q E300Q
P278R
A491P
R334G R239G R310G
S62R
S154R S341R S309R S278R S313R
V333V
G346A
T624R T583R
L593V L649V
P548A P534A P513A P448A P523A
R30R
T79P
S234P
A333P
N111D
A823G A615G
S763R
L347V L296V
G116R
N455K N466K N448K
S279T S282T
V607G V633G V623G
P205A
A910G
Q556E Q537E
T766R
S20T
C401S
L230V L309V
L402L
A130G A425G A434G
G1094G
R470G
A221G
V198V
A150G
A318G A302G
R1235P
G961A G944A G

  0%|          | 6/2546 [00:01<08:05,  5.23it/s]

R875G
C711S
L91V
R421G R540G
R287P R194P R178P R291P
S370R
S291T
D998E D1082E
A851P A850P
S396T
R1214P
D53G
R740T
F467L
D257H D43H
V587L
M36I
D1068H
E1703D E1708D
S463C
R1113G R1075G
K86N
L234L L383L
H145Q H196Q
G1022A G1026A G1035A G1013A
A7G
A47P
S374C
L373V L297V
R114R
W418C W391C
A3G
D193H D143H D175H D150H D208H D148H
S38T
A309P
S329S
L26V
G19G
V611G V565G V593G V575G V562G
Q1295H
Q791H
L411V
G628G G669G
S641C
Q293E
P483A
C428S C416S
R1549T R1595T
L515L
E796Q E836Q E765Q
G314A G246A
R422G R408G R298G
A463P A422P A409P A350P
P319A
T201P
L867V
R912P R823P
Q11H
A508P A507P
L387V
D267E
S19T
A34G
W48C
L489V L287V L451V
P732P
L2200V L2226V
L1591V
A185G
A206A
R385P
G119G
G112R
A691P A686P
R308R
M185I
P60R
P3079P P3083P
L1251V L2860V L2804V L1195V
L60L
K953N
A1570P
A350P
A314P
S128T
K215N K173N
A161P A441P A336P
S196T S340T S348T S195T
A421P
Y406X Y457X
D299E D137E D261E
T290A
L264V
L108V
V79G
A258A
C264S C230S
C7W
G578R
G150R G136R
D73E
T770R
A236G A273G A274G A304G A303G
T737T
Y41X
Q114

  0%|          | 8/2546 [00:01<10:15,  4.12it/s]

...

G65R
D738D
C758C
V49V
E192E
A7V
K874fs
A311T
G270G
L335F L235F L185F
R8T
A1144G A1174G
P857A P897A P836A P423A
G1060A
D446E D376E
A570P
G489A G469A G532A
G1239A
P1258R P1211R
S13R S485R
L586F L585F
G80G
T237P
A195P
A63G A46G
H274D
P111A P351A P367A P139A P275A P395A P427A P85A P350A
R5P
T5P
E420D
A160P A58P
D76H D53H
L304V
A73P A102P
R658P R948P R1232P R617P R437P R1043P
R87T R85T
H1130D H1086D H1140D H1076D
R84G
A410P
A147G A188G
Q84H Q119H Q109H Q130H Q144H
L257V
E199D E242D
E791Q E583Q
D382E
R237T R17T
G787A
A290P
G722R
R337P R33P R328P
K547N
S41T
G100G G178G
S1354R S1307R S1294R S1360R
N343K
S147C
P666R
T756P T755P T727P T701P T784P T700P T783P T728P T773P T699P T809P T774P T673P
L335V L342V
E1434Q E1450Q
Q588H
L781L
R52T
L1805V
L626L
T311S
A157P A153P
P1707A P1609A P1602A
R1593R
S22S
K179N K153N
A38P
P24A
Q60H
V555L V572L
L402V L293V L369V L192V L404V L291V
V898V V908V V797V
L260L L223L L64L
G53G G25G
D202E D91E
E3893Q
R213G
A855P
D67H
V5L
F98L F507L
V648V V606V
V61V
Q170E
A140P A

100%|█████████▉| 2537/2546 [03:30<00:00,  9.45it/s]

S425T S570T
R6P
N299K
A347G A319G
L91V
A266G A261G
R255T
S1067S
P3049A P1440A P1496A P3105A
S479X S486X
G841G
A1570G
Q187H
R646G R629G R628G R327G R645G R587G
A169P
L681V
L450V
L364L
R474S R423S
A1102P
G505G G378G G546G G419G
A8P
A374P
L135L
D590H
P50R P64R
G445A
A406P
L827V L783V
S161R
A3P
N99K
L256V
Q335H
V97V
Q3457H
A186P
T506S T512S
S11T
S381X S357X
G1108A G782A G729A
V245G
G1187G G1154G G1197G G1188G G1251G G1134G G1163G G1231G G1184G G1193G G1248G G1225G G1009G G1107G G1204G G1236G G1249G G1271G G1247G G1207G G1265G G1206G G1202G G1196G G1213G G1250G G1226G G1227G G1269G G1150G G1140G G1222G G1218G G1211G G1177G G1168G G1058G G1136G G1205G G1230G G1208G G1245G G1137G
K187N
L159L L193L
V111L
G528G
Q1002E
A169P A302P
A931G A805G
L308L L272L
E1117D
K400E
S234T
R442P
V43V
E271G E103G E234G E212G E249G E155G
V352V
E213D
S567T S633T S183T S241T S558T S212T S576T
L2V
L426V L531V L483V
A652G A598G
P279A
I616M
A43G A89G
A685G
G686R
A180G A4G
R157P R175P R68P
A149G A198G A204G
E59Q
G1649R


100%|██████████| 2546/2546 [03:31<00:00, 12.06it/s]

Q1288K
R894*
Y558C
T1046M
A1117A
V595I
S53T L43R
E152E
H641Q
W26*
T86S
Q644Sfs Q580Sfs Q634Sfs Q590Sfs
H58Y
R36H
R80*
L217I
P221P P251P
R5M
G606D
I248Nfs
R1517H
T1363P
S46S
W288fs
L359fs


In [ ]:
len(t_genetic_variation_fs_stop)

1944

In [ ]:
test.head()

In [ ]:
test.isnull().sum().sum()

237

## WT or 동의돌연변이 로만 구성되어있는 row가 어떤 subclass에 분포되어있는지


In [18]:
data_distribution_same_amino

{'KIPAN': 8,
 'SARC': 6,
 'SKCM': 0,
 'KIRC': 4,
 'GBMLGG': 4,
 'STES': 4,
 'BRCA': 18,
 'THCA': 50,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 2,
 'OV': 0,
 'PRAD': 16,
 'UCEC': 0,
 'LAML': 34,
 'COAD': 0,
 'ACC': 0,
 'LGG': 6,
 'LUSC': 4,
 'LUAD': 0,
 'CESC': 6,
 'PCPG': 12,
 'THYM': 68,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

test set에도 wt only가 있는지

In [ ]:
test

In [35]:
t_genetic_variation_same_amino = []
t_data_distribution_same_amino = {}
for i in range(len(cancer)):
    t_data_distribution_same_amino[cancer[i]] = 0
t_data_distribution_same_amino

{'KIPAN': 0,
 'SARC': 0,
 'SKCM': 0,
 'KIRC': 0,
 'GBMLGG': 0,
 'STES': 0,
 'BRCA': 0,
 'THCA': 0,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 0,
 'OV': 0,
 'PRAD': 0,
 'UCEC': 0,
 'LAML': 0,
 'COAD': 0,
 'ACC': 0,
 'LGG': 0,
 'LUSC': 0,
 'LUAD': 0,
 'CESC': 0,
 'PCPG': 0,
 'THYM': 0,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

In [36]:
ck_list = []
for i in tqdm(range(len(test))):
    for j in range(1, len(data_columns)-1):
        flag_break=False
        if test.iloc[i, j] == "WT":
            continue
        else:
            if " " not in test.iloc[i, j]:
                if '>' in test.iloc[i, j]:
                    ck_list.append(f"{i}, {j}")
                    continue

                elif test.iloc[i, j][0] != test.iloc[i, j][-1]:
                    flag_break=True
                    break

            else:
                amino = test.iloc[i, j].split()
                ch = 0
                for k in range(len(amino)):

                    if amino[k][0] == amino[k][-1]:

                        ch += 1
                    elif '>' in amino[k]:
                        ch += 1
                if (ch/len(amino)) != 1:
                    flag_break=True
                    break

    if flag_break:
        continue
    t_genetic_variation_same_amino.append(i)
    t_data_distribution_same_amino[train.iloc[i,1]] += 1


100%|██████████| 2546/2546 [00:30<00:00, 83.66it/s] 


In [25]:
t_genetic_variation_same_amino

[17,
 75,
 180,
 187,
 192,
 214,
 219,
 294,
 374,
 435,
 446,
 450,
 545,
 676,
 696,
 787,
 855,
 880,
 898,
 979,
 1023,
 1055,
 1249,
 1542,
 1690,
 1770,
 1785,
 1832,
 1866,
 1965,
 2046,
 2059,
 2118,
 2132,
 2153,
 2281,
 2340,
 2412,
 2481]

In [37]:
len(t_genetic_variation_same_amino)

39

In [38]:
len(t_genetic_variation_same_amino)

39

## train set에 >가 있는 row 와 value

In [49]:
ck_list = []
ck_row_list = []
for i in tqdm(range(len(train))):
    for j in range(2, len(data_columns)):
        flag_break=False
        if train.iloc[i, j] == "WT":
            continue
        else:
            if " " not in train.iloc[i, j]:
                if '>' in train.iloc[i, j]:
                    ck_list.append(f"{i}, {j}")
                    flag_break=True
                    break

            else:
                amino = train.iloc[i, j].split()
                for k in range(len(amino)):
                    if '>' in amino[k]:

                        ck_list.append(f"{i}, {j}")
                        flag_break=True

    if flag_break:
        ck_row_list.append(i)



100%|██████████| 6201/6201 [12:54<00:00,  8.01it/s]


In [50]:
ck_list

['14, 2525',
 '73, 2433',
 '90, 551',
 '111, 1580',
 '135, 2979',
 '149, 1196',
 '184, 3451',
 '188, 1633',
 '209, 2490',
 '248, 3538',
 '353, 3567',
 '363, 1923',
 '378, 4315',
 '401, 1732',
 '486, 1730',
 '495, 321',
 '505, 1936',
 '531, 1321',
 '532, 169',
 '582, 1049',
 '604, 279',
 '646, 3975',
 '650, 2142',
 '720, 3481',
 '722, 1020',
 '736, 1926',
 '753, 2256',
 '755, 229',
 '755, 2960',
 '779, 3605',
 '789, 2967',
 '798, 3850',
 '843, 3789',
 '861, 1381',
 '877, 3263',
 '880, 1858',
 '889, 3132',
 '917, 1388',
 '923, 1123',
 '946, 2056',
 '959, 875',
 '1014, 2009',
 '1035, 297',
 '1044, 3252',
 '1098, 937',
 '1109, 942',
 '1114, 3523',
 '1162, 1776',
 '1230, 3360',
 '1235, 1170',
 '1243, 3855',
 '1279, 3808',
 '1330, 2427',
 '1334, 3954',
 '1351, 755',
 '1362, 944',
 '1363, 1812',
 '1369, 770',
 '1369, 770',
 '1369, 1179',
 '1369, 1179',
 '1369, 2046',
 '1369, 2046',
 '1369, 2223',
 '1369, 2223',
 '1369, 2523',
 '1369, 2523',
 '1369, 3471',
 '1369, 3471',
 '1392, 2635',
 '1394,

In [52]:
len(ck_list)

197

In [51]:
ck_row_list

[73,
 90,
 111,
 135,
 149,
 184,
 188,
 209,
 248,
 353,
 363,
 378,
 401,
 486,
 495,
 505,
 531,
 532,
 582,
 604,
 646,
 650,
 722,
 736,
 753,
 755,
 779,
 789,
 798,
 843,
 861,
 877,
 880,
 889,
 917,
 923,
 946,
 959,
 1014,
 1035,
 1044,
 1098,
 1109,
 1114,
 1162,
 1230,
 1235,
 1243,
 1279,
 1330,
 1351,
 1362,
 1392,
 1394,
 1450,
 1496,
 1582,
 1600,
 1650,
 1662,
 1683,
 1693,
 1725,
 1774,
 1828,
 1850,
 1866,
 1883,
 1907,
 1996,
 2031,
 2092,
 2137,
 2148,
 2182,
 2206,
 2312,
 2322,
 2329,
 2379,
 2420,
 2436,
 2489,
 2552,
 2578,
 2656,
 2695,
 2759,
 2762,
 2771,
 2791,
 2969,
 3019,
 3059,
 3162,
 3206,
 3213,
 3237,
 3246,
 3372,
 3396,
 3405,
 3442,
 3483,
 3521,
 3581,
 3626,
 3661,
 3666,
 3726,
 3734,
 3735,
 3793,
 3929,
 3942,
 4097,
 4102,
 4144,
 4191,
 4213,
 4223,
 4234,
 4241,
 4347,
 4367,
 4409,
 4432,
 4588,
 4612,
 4625,
 4713,
 4729,
 4739,
 4773,
 4808,
 4864,
 4874,
 4963,
 4966,
 5068,
 5076,
 5194,
 5282,
 5318,
 5356,
 5446,
 5474,
 5504,
 554

In [53]:
len(ck_row_list)

164

## test set에 >가 있는 row 와 value

In [42]:
t_ck_list = []
t_ck_row_list = []
for i in tqdm(range(len(test))):
    for j in range(1, len(data_columns)-1):
        flag_break=False
        if test.iloc[i, j] == "WT":
            continue
        else:
            if " " not in test.iloc[i, j]:
                if '>' in test.iloc[i, j]:
                    t_ck_list.append(f"{i}, {j}")
                    flag_break=True


            else:
                amino = test.iloc[i, j].split()
                for k in range(len(amino)):
                    if '>' in amino[k]:

                        t_ck_list.append(f"{i}, {j}")
                        flag_break=True

    if flag_break:
        t_ck_row_list.append(i)



100%|██████████| 2546/2546 [05:23<00:00,  7.88it/s]


In [46]:
len(t_ck_list)

35

In [45]:
len(t_ck_row_list)

33

## 동의돌연변이 분석

### WT로만 되어있는 환자의 subclass 분포

In [6]:
train_WT = []
train_WT_distribution = {}
for i in range(len(cancer)):
    train_WT_distribution[cancer[i]] = 0
train_WT_distribution

{'KIPAN': 0,
 'SARC': 0,
 'SKCM': 0,
 'KIRC': 0,
 'GBMLGG': 0,
 'STES': 0,
 'BRCA': 0,
 'THCA': 0,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 0,
 'OV': 0,
 'PRAD': 0,
 'UCEC': 0,
 'LAML': 0,
 'COAD': 0,
 'ACC': 0,
 'LGG': 0,
 'LUSC': 0,
 'LUAD': 0,
 'CESC': 0,
 'PCPG': 0,
 'THYM': 0,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

In [9]:
for i in tqdm(range(len(train))):
    for j in range(2, len(data_columns)):
        if train.iloc[i, j] != "WT":
            break
        if j == len(data_columns)-1:

            train_WT.append(i)
            train_WT_distribution[train.iloc[i,1]] += 1

100%|██████████| 6201/6201 [01:27<00:00, 70.89it/s]


In [11]:
len(train_WT)

94

In [10]:
train_WT_distribution

{'KIPAN': 4,
 'SARC': 3,
 'SKCM': 0,
 'KIRC': 2,
 'GBMLGG': 2,
 'STES': 2,
 'BRCA': 5,
 'THCA': 18,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 1,
 'OV': 0,
 'PRAD': 4,
 'UCEC': 0,
 'LAML': 12,
 'COAD': 0,
 'ACC': 0,
 'LGG': 3,
 'LUSC': 2,
 'LUAD': 0,
 'CESC': 3,
 'PCPG': 3,
 'THYM': 30,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

### 동의돌연변이(ex. ROOOR) 되어있는 환자의 subclass 분포

In [19]:
list(data_distribution_same_amino.keys())

['KIPAN',
 'SARC',
 'SKCM',
 'KIRC',
 'GBMLGG',
 'STES',
 'BRCA',
 'THCA',
 'LIHC',
 'HNSC',
 'PAAD',
 'OV',
 'PRAD',
 'UCEC',
 'LAML',
 'COAD',
 'ACC',
 'LGG',
 'LUSC',
 'LUAD',
 'CESC',
 'PCPG',
 'THYM',
 'BLCA',
 'TGCT',
 'DLBC']

In [20]:
train_same_distribution = {}

for x in list(data_distribution_same_amino.keys()):

    train_same_distribution[x] = data_distribution_same_amino[x]-train_WT_distribution[x]
train_same_distribution

{'KIPAN': 0,
 'SARC': 0,
 'SKCM': 0,
 'KIRC': 0,
 'GBMLGG': 0,
 'STES': 0,
 'BRCA': 4,
 'THCA': 7,
 'LIHC': 0,
 'HNSC': 0,
 'PAAD': 0,
 'OV': 0,
 'PRAD': 4,
 'UCEC': 0,
 'LAML': 5,
 'COAD': 0,
 'ACC': 0,
 'LGG': 0,
 'LUSC': 0,
 'LUAD': 0,
 'CESC': 0,
 'PCPG': 3,
 'THYM': 4,
 'BLCA': 0,
 'TGCT': 0,
 'DLBC': 0}

### 동의돌연변이(ex. OOO_OOO>PL) 되어있는 환자의 subclass 분포
- 존재 하지 않음